In [1]:
import pandas as pd
import time
from datetime import datetime
from alpha_vantage.timeseries import TimeSeries
import random
import urllib
import sqlalchemy
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
import keras
import numpy as np
import pickle

position = {}
remaining_cash = 10000.00
capital = 10000.00
loss = 0
win = 0
profit = 0
commission = 5
max_profit = 0
min_loss = 0

C:\Users\Bikal\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
params = urllib.parse.quote("DRIVER={ODBC Driver 13 for SQL Server};SERVER=localhost;DATABASE=StocksDMW;Trusted_Connection=yes")
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


In [3]:
symbols = """
select distinct(name) from stockdatadaily order by name
"""
symbol_list = pd.read_sql(symbols, engine)

In [4]:
symbol_list = symbol_list['name'].tolist()

<h1> Instantation of variables

In [5]:
class equity():
    bought = None
    support = None
    resistance = None
    resistance_date = None
    stop_order = 0.0
    times_bought = 0
    symbol = None
    dataframe = None
    current_price = None
    flag_for_wins = False
    
    def __init__(self, name):
        self.symbol = name
        self.training = list()

sql1= """
select distinct(name) from stockdatadaily order by name
"""
names = pd.read_sql(sql1,engine)['name'].tolist()

stock_equities = {}
for name in names:
    stock_equities[name] = equity(name)



<h1> Updating resources

In [6]:
cursor = len(symbol_list)
percent = cursor // 100


counter = 0
percentage = 0
for symbol in symbol_list:
    if(counter % percent == 0 ):
        print(percentage , "% done")
        percentage += 1
    symbol_data = """
    select * from stockdatadaily where name = '%s'
    """ % symbol
    counter += 1
    stock_equities[symbol].dataframe = pd.read_sql(symbol_data,engine)

0 % done
1 % done
2 % done
3 % done
4 % done
5 % done
6 % done
7 % done
8 % done
9 % done
10 % done
11 % done
12 % done
13 % done
14 % done
15 % done
16 % done
17 % done
18 % done
19 % done
20 % done
21 % done
22 % done
23 % done
24 % done
25 % done
26 % done
27 % done
28 % done
29 % done
30 % done
31 % done
32 % done
33 % done
34 % done
35 % done
36 % done
37 % done
38 % done
39 % done
40 % done
41 % done
42 % done
43 % done
44 % done
45 % done
46 % done
47 % done
48 % done
49 % done
50 % done
51 % done
52 % done
53 % done
54 % done
55 % done
56 % done
57 % done
58 % done
59 % done
60 % done
61 % done
62 % done
63 % done
64 % done
65 % done
66 % done
67 % done
68 % done
69 % done
70 % done
71 % done
72 % done
73 % done
74 % done
75 % done
76 % done
77 % done
78 % done
79 % done
80 % done
81 % done
82 % done
83 % done
84 % done
85 % done
86 % done
87 % done
88 % done
89 % done
90 % done
91 % done
92 % done
93 % done
94 % done
95 % done
96 % done
97 % done
98 % done
99 % done
100 % done

In [7]:
sql_dates = """
select distinct(date) from stockdatadaily order by date
"""
dates = pd.read_sql(sql_dates, engine)['date'].tolist()

<h1> Loading KNN model </h1>

In [8]:
# model = tf.keras.models.load_model('Best1.model')
pickle_in = open("KNN-DailyModel63_46days.pickle","rb")
model = pickle.load(pickle_in)

<h1> Trading strategy with the training sets generation

In [9]:
def automatic_trader(symbol_list):
    global position
    global remaining_cash
    global loss
    global profit
    global win
    global max_profit
    global min_loss
    global commission
    
    for date in dates:
        symbols_in_date = """
        select distinct(name) from stockdatadaily where date = '%s'
        """ % date
        symbols = pd.read_sql(symbols_in_date,engine)['name'].tolist()
        for symbol in symbol_list:
            if symbol in symbols:
                current_symbol = stock_equities[symbol]
                data_till_date = data_extract(current_symbol.dataframe,date)
                current_price = data_till_date.tail(1)['2. high'].values[0]
                current_symbol.current_price = current_price
                if len(data_till_date) == 1:
                    continue
                if current_symbol.bought == None:
                    if resist_condition(data_till_date):
                        current_symbol.resistance, current_symbol.resistance_date = resistance_cal(data_till_date)
                        
                    if current_symbol.resistance != None and support_condition(data_till_date, current_symbol.resistance):
                        current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                    
                    if current_symbol.support != None and current_symbol.resistance !=None and buy_condition(current_symbol.resistance, data_till_date):
                        training_inst = training_set_extractor(data_till_date, date)
                        training_inst = np.asarray(training_inst)
                        for_predict = np.append([tf.keras.utils.normalize(training_inst[0])], [tf.keras.utils.normalize(training_inst[1])])
                        if(model.predict([for_predict]) == 1):
                            current_symbol.bought = current_price
                            current_symbol.stop_order = current_symbol.support * 0.97
                            quantity = (0.02 * capital) / current_price
                            remaining_cash = remaining_cash - commission - (quantity * current_price)                        
                            position[symbol]=quantity
                            current_symbol.times_bought += 1
                            print("Buying ", quantity, "stocks of ", symbol, "At price", current_symbol.bought, "on date", date)
    
                            
                else:
                    if current_symbol.flag_for_wins == False and current_symbol.stop_order > current_symbol.bought:
                                
                                win += 1
                                current_symbol.flag_for_wins = True
                                
                        
                        
                    if current_symbol.stop_order > current_price:                
                        current_symbol.stop_order = None
                        print("Selling ",position[symbol], "stocks of", symbol, " at price", current_price, "on date", date)
                        sold = current_price * position[symbol] - commission
                        profit_today = sold - current_symbol.bought * position[symbol]
                        if profit_today > 0:
                            
                            if current_symbol.flag_for_wins == False:
                                win += 1
                                
                            current_symbol.flag_for_wins = False
                            
                        else:
                            loss += 1
                            
                            current_symbol.flag_for_wins = False
                    
                            
                        profit += profit_today
                        
                        if profit > max_profit:
                            max_profit = profit
                        if profit < min_loss:
                            min_loss = profit
                        
                        remaining_cash = remaining_cash + sold
                        del position[symbol]
                        current_symbol.bought = None
                        
                    else:
                        if resist_condition(data_till_date):
                            current_symbol.resistance, current_symbol.resistance_date = resistance_cal( data_till_date)
                        
                        if current_symbol.resistance and support_condition(data_till_date,current_symbol.resistance):
                            current_symbol.support = support_cal(current_symbol.resistance_date, data_till_date)
                        
                        if current_symbol.resistance != None and current_symbol.support != None and buy_condition(current_symbol.resistance, data_till_date):
                            current_symbol.stop_order = current_symbol.support * 0.97
                    
                        
    print("Win Percentage")            
    print((win/(loss + win))*100)
    print("Profit",profit)
    print("\n")                    

               
                
                




<h1> Helper functions for the strategy

In [10]:
def resistance_cal(data_resistance):

    highest_data = data_resistance[data_resistance['2. high'] == max(data_resistance['2. high'])]
    date = highest_data['date'].values[0]
    resistance = highest_data['2. high'].values[0]


    return round(resistance + 0.005,2), date                

def support_cal(resist_date, data_support):

    # data since the last resistance date until today's date.
    data_since_last_resistance = data_support.loc[data_support['date'] >= resist_date]
    # data variable here contains the row of the dataframe with the lowest value of '3. low' values
    data = data_since_last_resistance.loc[data_since_last_resistance['3. low']==min(data_since_last_resistance['3. low'])]
    # support here contains the lowest value of stock since last resistance date
    support = data['3. low'].values[0]


    # we return the support value and the date of the support value
    return support

def resist_condition(  data_resistance):

    recent_data = data_resistance.tail(2)   # Getting yesterday's and today's price
    yesterday_price = recent_data['3. low'].values[0]   # Yesterday's prices
    today_price = recent_data['3. low'].values[1]       # today's prices  


    if(today_price < yesterday_price):
        return True
    else:
        return False
    # If today's lowest price of the stock is less than yesterday's lowest price of the stock,
    # then Resistance condition is met

def support_condition(   data_support, resist_passed):
    current_price = data_support.tail(1)['2. high'].values[0] # Today's price of the stock
    if(current_price > resist_passed):
        return True
    else:
        return False
    #If the current price of the stock is greater than the current resistance value that is passed then,
    # the support condition is met     

def buy_condition(resist, data):
    if resist < data.tail(1)['2. high'].values[0]:
        return True
    else:
        return False       

def data_extract(data, *args):
    if len(args) == 2:
        data = data.loc[data['date'] >= args[0]]
        data = data.loc[data['date'] <= args[1]]
    else:
        data = data.loc[data['date'] <= args[0]]
    return data

def training_set_extractor(data, date):
    lastFrom = pd.to_datetime(date)

    destDate = lastFrom - pd.Timedelta(days=45)
    lastFrom = str(lastFrom.date())
    destDate = str(destDate.date())

    df = pd.DataFrame()
    df = data[(data['date'] >= destDate) & (data['date'] <= lastFrom)]

    df = df.set_index('date')

    df.index = pd.DatetimeIndex(df.index)
    idx = pd.date_range(destDate, lastFrom)
    df = df.reindex(idx, fill_value=0)
    
    highs = df['2. high'].tolist()
    volumes = df['5. volume'].tolist()
    lows = df['3. low'].tolist()
    
    for i in range(1, len(highs)):
        if highs[i] == 0:
            highs[i] = highs[i-1]
        if volumes[i] == 0:
            volumes[i] = volumes[i-1]
        if lows[i] == 0:
            lows[i] = lows[i-1]
        
    
    training_instance = [highs, volumes, lows, 'na']
    
    return training_instance[:] 
   

In [11]:
automatic_trader(symbol_list)

Buying  18.501387604070306 stocks of  CNX At price 10.81 on date 2000-01-06
Buying  30.473868657626088 stocks of  CXE At price 6.563 on date 2000-01-06
Buying  7.047216349541931 stocks of  CYBE At price 28.38 on date 2000-01-06
Buying  5.0 stocks of  EMKR At price 40.0 on date 2000-01-06
Buying  34.07155025553663 stocks of  FCN At price 5.87 on date 2000-01-06
Buying  56.64117813650524 stocks of  GIII At price 3.531 on date 2000-01-06
Buying  9.246417013407305 stocks of  HP At price 21.63 on date 2000-01-06
Buying  3.3057851239669422 stocks of  LIFE At price 60.5 on date 2000-01-06
Buying  8.841732979664014 stocks of  MCY At price 22.62 on date 2000-01-06
Buying  31.68065895770632 stocks of  OCN At price 6.313 on date 2000-01-06
Buying  7.183908045977011 stocks of  XLE At price 27.84 on date 2000-01-06
Buying  47.05882352941177 stocks of  AMAG At price 4.25 on date 2000-01-10
Buying  6.709158000670916 stocks of  FLIC At price 29.81 on date 2000-01-14
Buying  42.10526315789474 stocks of

Buying  72.72727272727273 stocks of  ATRS At price 2.75 on date 2000-01-24
Buying  38.095238095238095 stocks of  AWX At price 5.25 on date 2000-01-24
Buying  9.066183136899365 stocks of  AXE At price 22.06 on date 2000-01-24
Buying  6.765899864682003 stocks of  BDC At price 29.56 on date 2000-01-24
Buying  15.384615384615385 stocks of  CCA At price 13.0 on date 2000-01-24
Buying  1.9912385503783354 stocks of  CCZ At price 100.44 on date 2000-01-24
Buying  4.705882352941177 stocks of  CERS At price 42.5 on date 2000-01-24
Buying  57.142857142857146 stocks of  CLBS At price 3.5 on date 2000-01-24
Buying  14.316392269148174 stocks of  COBZ At price 13.97 on date 2000-01-24
Buying  6.25 stocks of  DRQ At price 32.0 on date 2000-01-24
Buying  25.094102885821833 stocks of  ELGX At price 7.97 on date 2000-01-24
Buying  8.695652173913043 stocks of  ESBK At price 23.0 on date 2000-01-24
Buying  3.3261267254282387 stocks of  FORTY At price 60.13 on date 2000-01-24
Buying  72.72727272727273 stock

Buying  9.46521533364884 stocks of  CLB At price 21.13 on date 2000-01-27
Buying  8.421052631578947 stocks of  CLFD At price 23.75 on date 2000-01-27
Selling  7.766990291262136 stocks of ELS  at price 24.38 on date 2000-01-27
Buying  32.98697014679202 stocks of  MITK At price 6.063 on date 2000-01-27
Buying  23.880597014925375 stocks of  TACT At price 8.375 on date 2000-01-27
Buying  13.670539986329459 stocks of  VTN At price 14.63 on date 2000-01-27
Buying  28.571428571428573 stocks of  ACY At price 7.0 on date 2000-01-28
Buying  13.793103448275861 stocks of  ARDM At price 14.5 on date 2000-01-28
Buying  87.71929824561404 stocks of  CYAN At price 2.28 on date 2000-01-28
Buying  11.229646266142618 stocks of  EOG At price 17.81 on date 2000-01-28
Buying  4.705882352941177 stocks of  EXC At price 42.5 on date 2000-01-28
Buying  62.131096613855235 stocks of  KOOL At price 3.219 on date 2000-01-28
Buying  21.186440677966104 stocks of  LCII At price 9.44 on date 2000-01-28
Buying  11.806375

Buying  7.6923076923076925 stocks of  BLX At price 26.0 on date 2000-02-08
Buying  50.994390617032124 stocks of  CALM At price 3.922 on date 2000-02-08
Buying  5.903187721369539 stocks of  CCU At price 33.88 on date 2000-02-08
Buying  59.25925925925926 stocks of  DAIO At price 3.375 on date 2000-02-08
Buying  5.298013245033113 stocks of  DSGX At price 37.75 on date 2000-02-08
Buying  57.142857142857146 stocks of  EDUC At price 3.5 on date 2000-02-08
Buying  133.33333333333334 stocks of  ENG At price 1.5 on date 2000-02-08
Buying  18.6046511627907 stocks of  EVJ At price 10.75 on date 2000-02-08
Buying  13.333333333333334 stocks of  FEIM At price 15.0 on date 2000-02-08
Buying  14.285714285714286 stocks of  LAQ At price 14.0 on date 2000-02-08
Buying  26.229508196721312 stocks of  LCUT At price 7.625 on date 2000-02-08
Buying  17.196904557179707 stocks of  LDF At price 11.63 on date 2000-02-08
Buying  8.554319931565441 stocks of  MAC At price 23.38 on date 2000-02-08
Buying  23.70229912

Selling  4.155412424683149 stocks of BA  at price 38.69 on date 2000-02-15
Buying  89.52551477170994 stocks of  CYTR At price 2.234 on date 2000-02-15
Buying  266.6666666666667 stocks of  EFOI At price 0.75 on date 2000-02-15
Buying  6.956521739130435 stocks of  ESV At price 28.75 on date 2000-02-15
Buying  6.896551724137931 stocks of  LFUS At price 29.0 on date 2000-02-15
Buying  1.4414414414414414 stocks of  MS At price 138.75 on date 2000-02-15
Buying  15.6128024980484 stocks of  MYC At price 12.81 on date 2000-02-15
Buying  9.090909090909092 stocks of  O At price 22.0 on date 2000-02-15
Buying  4.848484848484849 stocks of  SKM At price 41.25 on date 2000-02-15
Buying  17.482517482517483 stocks of  USA At price 11.44 on date 2000-02-15
Buying  4.4257579110422665 stocks of  AMD At price 45.19 on date 2000-02-16
Selling  3.0623181748583677 stocks of AVX  at price 64.0 on date 2000-02-16
Buying  53.333333333333336 stocks of  BREW At price 3.75 on date 2000-02-16
Selling  5.903187721369

...

Buying  4.894762604013706 stocks of  GUSH At price 40.86 on date 2015-06-18
Buying  4.6317739694302915 stocks of  DRIP At price 43.18 on date 2015-06-19
Selling  12.836970474967908 stocks of AMLP  at price 16.26 on date 2015-06-22
Buying  3.5192679922576104 stocks of  EDEN At price 56.83 on date 2015-06-22
Selling  95.23809523809524 stocks of GPL  at price 0.433 on date 2015-06-22
Buying  4.081632653061225 stocks of  INDB At price 49.0 on date 2015-06-22
Buying  7.751937984496124 stocks of  NGHCO At price 25.8 on date 2015-06-22
Selling  111.73184357541899 stocks of OHRP  at price 2.5999 on date 2015-06-22
Buying  9.191176470588236 stocks of  SPUN At price 21.76 on date 2015-06-22
Selling  4.404316229905308 stocks of TGP  at price 34.16 on date 2015-06-22
Buying  2.684563758389262 stocks of  THG At price 74.5 on date 2015-06-22
Selling  6.715916722632639 stocks of YELP  at price 46.02 on date 2015-06-22
Selling  2.7405143945518575 stocks of CSM  at price 50.79 on date 2015-06-26
Buying

Buying  5.446623093681917 stocks of  INBK At price 36.72 on date 2015-08-10
Selling  7.462686567164179 stocks of KNOP  at price 15.245 on date 2015-08-10
Selling  6.049606775559588 stocks of KR  at price 38.24 on date 2015-08-10
Selling  14.646435057706954 stocks of LXFR  at price 12.49 on date 2015-08-10
Selling  11.49425287356322 stocks of NWSA  at price 14.38 on date 2015-08-10
Selling  9.103364148220976 stocks of QQQC  at price 22.9999 on date 2015-08-10
Selling  9.049773755656108 stocks of SMLP  at price 24.67 on date 2015-08-10
Buying  2.343017806935333 stocks of  STMP At price 85.36 on date 2015-08-10
Selling  6.946856547412296 stocks of TPZ  at price 23.05 on date 2015-08-10
Selling  7.5046904315197 stocks of ASHR  at price 41.39 on date 2015-08-11
Selling  11.389521640091116 stocks of CALA  at price 6.62 on date 2015-08-11
Selling  7.000350017500875 stocks of CLDT  at price 24.788 on date 2015-08-11
Selling  10.314595152140278 stocks of CPTA  at price 14.8699 on date 2015-08-1

Selling  1.7164435290078957 stocks of PII  at price 129.68 on date 2015-09-15
Selling  5.131257568604914 stocks of RFV  at price 50.11 on date 2015-09-15
Selling  4.223864836325237 stocks of VRNT  at price 47.24 on date 2015-09-15
Selling  7.942811755361398 stocks of WBIF  at price 23.0 on date 2015-09-15
Selling  7.683442182097579 stocks of ACWF  at price 23.2 on date 2015-09-16
Selling  9.852216748768473 stocks of ARGT  at price 18.005 on date 2015-09-16
Selling  1.8518518518518519 stocks of DIN  at price 93.77 on date 2015-09-16
Selling  7.407407407407407 stocks of HSEB  at price 25.52 on date 2015-09-16
Buying  6.540222367560498 stocks of  DERM At price 30.58 on date 2015-09-17
Buying  6.040471156750226 stocks of  FSV At price 33.11 on date 2015-09-17
Selling  39.52569169960475 stocks of OMEX  at price 0.4 on date 2015-09-18
Selling  2.9429508968642857 stocks of MSM  at price 65.045 on date 2015-09-21
Selling  2.38038562247084 stocks of AMGN  at price 146.78 on date 2015-09-22
Sell

Selling  3.0854674483184206 stocks of MGLN  at price 50.68 on date 2015-10-21
Selling  7.356186552890982 stocks of PHDG  at price 24.37 on date 2015-10-21
Selling  8.1799591002045 stocks of TBPH  at price 14.25 on date 2015-10-21
Buying  1.1089548100914888 stocks of  NOC At price 180.35 on date 2015-10-22
Buying  6.501950585175552 stocks of  INFO At price 30.76 on date 2015-10-23


ValueError: cannot reindex from a duplicate axis

In [ ]:
current_profit = 0
for key in position:
    current_profit = position[key] * (stock_equities[key].current_price - stock_equities[key].bought)
    remaining_cash += position[key] * stock_equities[key].current_price
    profit += current_profit
    
    if current_profit > 0:
        win += 1
    else:
        loss += 1
    
    if profit > max_profit:
        max_profit = profit
    
    if profit < min_loss:
        min_loss = profit
    
    
print("Maximum Profit is ", max_profit)
print("Minimum Loss is ", min_loss)
print("Profit", profit)
print("Remaining cash ", remaining_cash)
print("Win Percentage")            
print((win/(loss + win))*100)